# 다이소 재고 검색 프로젝트

- 기능 1. 해당 상품명 검색 시 추천순(or 판매량, 리뷰많은순) 상품 5개 선택
- 기능 2. 선택된 상품들에 대해서 리뷰 데이터 수집 (별점, 내용)
- 기능 3. 입력받은 지역 근처 다이소 매장 검색 (3개)
- 기능 4. 선택된 상품과 선택한 매장에 대해 재고 검색

> 입력: 상품명, 지역  
> 출력: 상품 리스트와 리뷰, 재고

다이소몰 URL: https://www.daisomall.co.kr/ds

### 입력 받기 & 필요한 모듈 임포트

In [3]:
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd

In [22]:
product, region = map(str, input('상품명과 지역을 입력해주세요(공백으로 구분)\n').split())

상품명과 지역을 입력해주세요(공백으로 구분)
텀블러 강남


### 해당 지역의 다이소 매장 찾는 함수

In [33]:
def get_store(region, browser):
    search_url = 'https://www.daisomall.co.kr/ms/msg/SCR_MSG_0019'
    browser.get(search_url)

    time.sleep(3) 
    
    # 매장 검색 박스
    search_box = browser.find_elements(by=By.CLASS_NAME, value='el-input__inner')[1] # index 0: product search box
    search_box.clear()
    search_box.send_keys(region)
    search_box.send_keys(Keys.ENTER)

    time.sleep(3)
    
    region_boxies = browser.find_elements(by=By.CLASS_NAME, value='store-map')

    store_list = []
    
    for i, region_box in enumerate(region_boxies):
        store = region_box.find_element(by=By.CLASS_NAME, value='tit-h5').text
        location = region_box.find_element(by=By.XPATH, value='//*[@id="pane-2"]/div/div[2]/div[{}]/div[2]/div[2]'.format(i+1)).text
        info = region_box.find_element(by=By.CLASS_NAME, value='info-group')
        opening = info.find_elements(by=By.TAG_NAME, value='span')[0].text.replace('영업 시간 ', '')
        tel = info.find_elements(by=By.TAG_NAME, value='span')[1].text.replace('매장 전화 ', '')
        
        time.sleep(1)
    
        store_list.append({'store':store.split()[0], 'location':location, 'opening':opening, 'tel':tel})
        
        store_df = pd.DataFrame(store_list)
        
    return store_df

### 메인

In [34]:
# 브라우저 옵션 설정
options = webdriver.EdgeOptions()
options.add_argument('--no-sandbox')                    
options.add_argument('--disable-dev-shm-usage')    

driver = webdriver.Edge(options = options)
driver.maximize_window()

# 함수 실행
result = get_store('명동', driver)

# 드라이버 종료
driver.quit()

In [35]:
result

,store,location,opening,tel
0,명동본점,서울특별시 중구 명동길 43 (명동1가),10:00 ~ 22:00,1522-4400
1,명동역점,서울특별시 중구 퇴계로 134-1(남산동3가),10:00 ~ 22:00,1522-4400
2,춘천명동점,강원도 춘천시 명동길 3-1 (조양동),10:00 ~ 22:00,1522-4400
3,부산화명점,부산광역시 북구 금곡대로 358 (화명동),10:00 ~ 22:00,1522-4400
4,청주봉명점,충청북도 청주시 흥덕구 덕암로30번길 8(봉명동),10:00 ~ 22:00,043-263-7776
5,광명사거리점,경기도 광명시 광명로 907 (광명동),09:30 ~ 22:00,1522-4400
6,대구안지랑점,대구광역시 남구 두류공원로 45(대명동),10:00 ~ 22:00,053-657-6502
7,대전한밭대점,대전광역시 유성구 학하서로121번길 55-13 (덕명동),10:00 ~ 22:00,042-825-5990
8,앞산순환로점,대구광역시 남구 앞산순환로 535 (대명동),10:00 ~ 22:00,053-652-6016
9,영대병원역점,대구광역시 남구 대명로 297 (대명동) 1층,10:00 ~ 22:00,053-626-5665


---

### 상품명과 매장명을 통한 재고 검색 함수

In [1]:
product, store_name = '텀블러', '명동본점'

In [ ]:
# 브라우저 옵션 설정
options = webdriver.EdgeOptions()
options.add_argument('--no-sandbox')                    
options.add_argument('--disable-dev-shm-usage')    

driver = webdriver.Edge(options = options)
driver.maximize_window()


In [22]:
def get_stock(product_no, store, browser):
    url = 'https://www.daisomall.co.kr/ms/msg/SCR_MSG_0019'
    browser.get(url)
    time.sleep(2)
    
    # step 0. 광고 닫기
    btn = browser.find_element(by=By.XPATH, value='//*[@id="btn-close-nday"]')
    btn.click()
    time.sleep(1)
    
    # step 1. 매장 상품 찾기 클릭
    btn = browser.find_element(by=By.XPATH, value='//*[@id="tab-3"]/button')
    btn.click()
    time.sleep(1)
    
    # step 2. 특정 상품 검색 및 선택
    search_box = browser.find_element(by=By.XPATH, value='//*[@id="pane-3"]/div/div[2]/div[1]/form/div/div/div/div/input')
    search_box.clear()
    search_box.send_keys(product_no)
    search_box.send_keys(Keys.ENTER)
    time.sleep(1)
    
    search_btn = browser.find_element(by=By.XPATH, value='//*[@id="__layout"]/section/div[1]/div/div[3]/div/div[2]/div/div[1]/button')
    search_btn.click()
    time.sleep(1)
    
    check_btn = browser.find_element(by=By.XPATH, value='//*[@id="__layout"]/section/div[1]/div/div[3]/div/div[2]/div/div[2]/div[1]/div[1]/label')
    check_btn.click()
    time.sleep(1)
    
    choice_btn = browser.find_element(by=By.XPATH, value='//*[@id="__layout"]/section/div[1]/div/div[3]/div/div[3]/div/button')
    choice_btn.click()
    time.sleep(1)
    
    # step 3. 해당 매장 검색 및 선택
    search_box = browser.find_element(by=By.XPATH, value='//*[@id="pane-3"]/div/div[2]/div[2]/form/div/div/div/div/div[1]/input')
    search_box.clear()
    search_box.send_keys(store)
    search_box.send_keys(Keys.ENTER)
    time.sleep(1)
    
    check_btn = browser.find_element(by=By.XPATH, value='//*[@id="__layout"]/section/div[1]/div/div[4]/div/div[2]/div[2]/div/label/span[1]/span')
    check_btn.click()
    time.sleep(1)
    
    choice_btn = browser.find_element(by=By.XPATH, value='//*[@id="__layout"]/section/div[1]/div/div[4]/div/div[3]/div/button')
    choice_btn.click()
    time.sleep(1)
    
    # step 4. 재고 정보 가져오기
    stock = browser.find_element(by=By.XPATH, value='//*[@id="pane-3"]/div/div[2]/div[3]/div/div[1]/div[4]/em[2]').text
    zone = browser.find_element(by=By.XPATH, value='//*[@id="pane-3"]/div/div[2]/div[3]/div/div[1]/div[4]/em[1]').text
    
    if stock[3:5] == '품절':
        stock = '품절'
    else:
        stock = ' '.join(stock.split()[1:3])
        
    zone = zone.split()[-1]
        
    return stock, zone
    

In [23]:
# test 
# >> input - product number

driver = webdriver.Edge()

s, z = get_stock('1036158', '명동본점', driver)

print(s, z)

driver.quit()

10개 이하 17


In [24]:
# test 
# >> input - product number

driver = webdriver.Edge()

s, z = get_stock('1036158', '부천본점', driver)

print(s, z)

driver.quit()

품절 23


---

### 유저 시나리오

In [ ]:
user_input1, user_input2 = map(int, input('찾을 상품과 현재 위치를 입력하세요').split())

# product_data -> 상품명, 가격, 링크, 평균별점
product_data = get_product_info(user_input)

# review_data -> 리뷰 텍스트, 별점
review_data = get_review(product_data['link'])

user_input3 = list(input('재고를 조회할 상품 번호를 선택해주세요(복수 선택 시 공백으로 구분)').split())

# store_data -> 매장명, 매장 위치
store_data = get_store(user_input2)

# inventory -> 해당 상품과 매장의 재고 정보
inventory = get_stock(product, store)